In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data_processed.csv")
df.head()

,sex,Age,Married,gained_asset,durable_asset,save_asset,living_expenses,other_expenses,incoming_salary,incoming_own_farm,...,11 family members,12 family members,2 family members,3 family members,4 family members,5 family members,6 family members,7 family members,8 family members,9 family members
0,1,28,1,28912201,22861940,23399979,26692283,28203066,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,23,1,28912201,22861940,23399979,26692283,28203066,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,22,1,28912201,22861940,23399979,26692283,28203066,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,27,1,52667108,19698904,49647648,397715,44042267,0,1,...,0,0,0,0,1,0,0,0,0,0
4,0,59,0,82606287,17352654,23399979,80877619,74503502,1,0,...,0,0,0,0,0,0,1,0,0,0


In [3]:
# Genero datos de entrenamiento y test
x = df.drop('depressed',axis=1) # Variable independiente
y = df['depressed'] # Variable dependiente

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23)

In [4]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

over_sampler = RandomOverSampler(random_state=42)
X_res, y_res = over_sampler.fit_resample(x_train, y_train)
print(f"Training target statistics: {Counter(y_res)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({0: 933, 1: 933})
Testing target statistics: Counter({0: 241, 1: 41})


# Random Forest

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

RandomForest = RandomForestClassifier()
RandomForest.fit(X_res.values,y_res.values)

RandomForestClassifier()

In [6]:
# from sklearn import tree
# fn=df.columns
# cn=df.columns
# fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
# tree.plot_tree(RandomForest.estimators_[0],
#                feature_names = fn, 
#                class_names=cn,
#                filled = True);
# fig.savefig('rf_individualtree.png')

In [7]:
Model = RandomForestClassifier(random_state=100,n_jobs=-1)

params = {'max_depth':[3,5,10,12,15],
         'min_samples_leaf':[60,70,80,90,100,120,130],
         'n_estimators':[200], 
         'max_features': [0.05,0.1,0.15,0.2],
         'criterion': ["gini"]}
grid_search = GridSearchCV(param_grid=params,estimator=Model,n_jobs=-1,verbose=1,scoring='recall')
grid_search.fit(X_res,y_res)

Fitting 5 folds for each of 140 candidates, totalling 700 fits


GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=100),
             n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [3, 5, 10, 12, 15],
                         'max_features': [0.05, 0.1, 0.15, 0.2],
                         'min_samples_leaf': [60, 70, 80, 90, 100, 120, 130],
                         'n_estimators': [200]},
             scoring='recall', verbose=1)

In [8]:
Model_best = grid_search.best_estimator_

In [9]:
Y_pred = RandomForest.predict(x_test.values) # test the output by changing values

In [10]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

acc_knn = metrics.accuracy_score(y_test,Y_pred)
acc_knn #0.8439716312056738

0.8156028368794326